In [7]:
import os
import pandas as pd
from dotenv import load_dotenv
import sqlalchemy
from sqlalchemy import update, MetaData, Table
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy.sql import func
load_dotenv()

True

In [10]:
engine = sqlalchemy.create_engine(f'mysql+pymysql://{os.getenv("DB_USER")}:{os.getenv("DB_PASS")}@{os.getenv("DB_HOST")}/{os.getenv("DB_NAME")}')
SessionLocal = sessionmaker(bind=engine)
session = SessionLocal()

In [27]:
# Reflect existing database tables
metadata = MetaData()
metadata.reflect(bind=engine)

In [36]:
# Define references to tables
news_table = Table('News', metadata, autoload_with=engine)
videos_table = Table('Videos', metadata, autoload_with=engine)
votes_table = Table('Votes', metadata, autoload_with=engine)
influencers_table = Table('Influencers', metadata, autoload_with=engine)

In [37]:
# Query data from News, Videos, and Votes tables
news_query = session.query(news_table).all()
videos_query = session.query(videos_table).all()
votes_query = session.query(votes_table).all()

In [40]:
Influencer = pd.read_sql_table('Influencers', engine)
Influencer.head()

,id,name,vibe_score,image_url
0,1,Taylor Swift,0.0,https://cdn.prod.website-files.com/61e1db1c178...
1,2,Kanye West,0.0,https://upload.wikimedia.org/wikipedia/commons...
2,3,MrBeast,0.0,https://upload.wikimedia.org/wikipedia/commons...
3,4,The Weeknd,0.0,https://upload.wikimedia.org/wikipedia/commons...
4,5,Justin Bieber,0.0,https://media.gettyimages.com/id/452816718/pho...


In [41]:
News = pd.read_sql_table('News', engine)
News.head()

,id,influencer_id,url,title,article,sentiment_score
0,1,1,https://www.tmz.com/2024/11/14/camila-cabello-...,Camila Cabello Announces 'Internet Break' Ahea...,Camila Cabellois signing off from the internet...,6.0
1,2,1,https://www.tmz.com/2024/11/13/pharrell-willia...,Pharrell Williams Denies Dissing Taylor Swift'...,Pharrell Williamscriticized music stars on bot...,5.0
2,3,1,https://www.tmz.com/2024/11/12/dean-cain-happy...,Dean Cain Glad Taylor Swift Fans Didn't Sway E...,Dean Cainis happyTaylor Swiftfans turned out t...,5.0
3,4,1,https://www.tmz.com/2024/11/12/fbi-investigati...,FBI Investigating Patrick Mahomes and Travis K...,The feds are working with local law enforcemen...,5.0
4,5,1,https://www.tmz.com/2024/11/12/patrick-mahomes...,Patrick Mahomes & Travis Kelce's Homes Burglar...,2:50 PM PT-- Sources familiar with the situati...,6.0


In [42]:
Videos = pd.read_sql_table('Videos', engine)
Videos.head()

,id,influencer_id,url,title,comment,sentiment_score
0,1,4,https://www.youtube.com/watch?v=bXBMv6j28-s,SAO PAULO // OUT NOW 🇧🇷,Deaf people:💀<br>Blind people:😄😃,5.0
1,2,4,https://www.youtube.com/watch?v=bXBMv6j28-s,SAO PAULO // OUT NOW 🇧🇷,bro what is this💀💀,5.0
2,3,4,https://www.youtube.com/watch?v=bXBMv6j28-s,SAO PAULO // OUT NOW 🇧🇷,Eyes In shock😮😵‍💫 but ears Mesmerized🎶👌💥,5.0
3,4,4,https://www.youtube.com/watch?v=bXBMv6j28-s,SAO PAULO // OUT NOW 🇧🇷,"We&#39;re all traumatized, thank you Abel 😄",5.0
4,5,4,https://www.youtube.com/watch?v=bXBMv6j28-s,SAO PAULO // OUT NOW 🇧🇷,Bro WHAT is this just drop the album Abel😭🙏🏽,5.0


In [43]:
Votes = pd.read_sql_table('Votes', engine)
Votes.head()

,id,influencer_id,good_vote,bad_vote
0,1,1,11,4
1,2,2,0,0
2,3,3,0,0
3,4,4,0,0
4,5,5,1,0


In [ ]:
# # Define the base for your models
# Base = declarative_base()

# # Influencer Table
# class Influencers(Base):
#     __tablename__ = 'Influencers'
#     id = Column(Integer, primary_key=True)
#     influencer_id = Column(Integer)
#     good_vote = Column(Integer)
#     bad_vote = Column(Integer)

# # News Table
# class News(Base):
#     __tablename__ = 'News'
#     id = Column(Integer, primary_key=True)
#     influencer_id = Column(Integer, ForeignKey('influencers.influencer_id'))
#     url = Column(String)
#     title = Column(String)
#     article = Column(String)
#     sentiment_score = Column(Float)

# # Video Table
# class Video(Base):
#     __tablename__ = 'Videos'
#     id = Column(Integer, primary_key=True)
#     influencer_id = Column(Integer, ForeignKey('influencers.influencer_id'))
#     url = Column(String)
#     title = Column(String)
#     comment = Column(String)
#     sentiment_score = Column(Float)

# # Votes Table
# class Vote(Base):
#     __tablename__ = 'Votes'
#     id = Column(Integer, primary_key=True)
#     influencer_id = Column(Integer)
#     good_vote = Column(Integer)
#     bad_vote = Column(Integer)

In [44]:
News = pd.DataFrame([row._asdict() for row in news_query])
Videos = pd.DataFrame([row._asdict() for row in videos_query])
Votes = pd.DataFrame([row._asdict() for row in votes_query])

In [45]:
# Define a function to calculate vote_score
def calculate_vote_score(good_vote, bad_vote):
    return (good_vote - bad_vote) / (good_vote + bad_vote + 1)

# Define a function to calculate normalized sentiment score (assuming sentiment_score is between 0-10)
def normalize_sentiment(sentiment_score):
    return sentiment_score / 10

# Define a function to calculate vibe score based on sentiment and vote scores
def calculate_vibe_score(news_sentiment, video_sentiment, vote_score):
    return round(0.5 * news_sentiment + 0.5 * video_sentiment + 0.5 * vote_score, 2)

In [26]:
average_news_sentiment = session.query(func.avg(News.sentiment_score)).scalar()
print(f'Average News Sentiment: {average_news_sentiment}')

Average News Sentiment: 5.2903


In [49]:
# Create a dictionary to store vibe scores for each influencer_id
vibe_scores = {}

# Iterate over each influencer in the Votes table
for index, vote_row in Votes.iterrows():
    influencer_id = vote_row['influencer_id']

    # Filter sentiment scores for the specific influencer
    filtered_scores = News[News['influencer_id'] == influencer_id]['sentiment_score']
    # Check if the filtered DataFrame is empty or if all values are NaN
    if filtered_scores.empty or filtered_scores.isna().all():
        avg_news_sentiment = 0.0  # Default to 0 if no valid scores exist
    else:
        avg_news_sentiment = filtered_scores.mean()  # Calculate the mean
    
    # # Fetch average sentiment scores for news articles related to this influencer
    # avg_news_sentiment = News[News['influencer_id'] == influencer_id]['sentiment_score'].mean() or 0
    
    # Fetch average sentiment scores for videos related to this influencer
    avg_video_sentiment = Videos[Videos['influencer_id'] == influencer_id]['sentiment_score'].mean() or 0
    
    # Normalize sentiment scores
    normalized_news_sentiment = normalize_sentiment(avg_news_sentiment)
    normalized_video_sentiment = normalize_sentiment(avg_video_sentiment)
    
    # Calculate vote score based on good and bad votes
    vote_score = calculate_vote_score(vote_row['good_vote'], vote_row['bad_vote'])
    
    # Calculate final vibe score
    vibe_score = calculate_vibe_score(normalized_news_sentiment, normalized_video_sentiment, vote_score)
    
    # Store the result in the dictionary with influencer_id as key and vibe score as value
    vibe_scores[influencer_id] = vibe_score

print(vibe_scores)

{np.int64(1): np.float64(0.84), np.int64(2): np.float64(0.55), np.int64(3): np.float64(0.5), np.int64(4): np.float64(0.53), np.int64(5): np.float64(0.85), np.int64(6): np.float64(0.57), np.int64(7): np.float64(0.54), np.int64(8): np.float64(0.56), np.int64(9): np.float64(0.46), np.int64(10): np.float64(0.6), np.int64(11): np.float64(0.62), np.int64(12): np.float64(0.57), np.int64(13): np.float64(0.6), np.int64(14): np.float64(0.58), np.int64(15): np.float64(0.56), np.int64(16): np.float64(0.49), np.int64(17): np.float64(0.54), np.int64(18): np.float64(0.5), np.int64(19): np.float64(0.76), np.int64(20): np.float64(0.53)}
